In [1]:
__author__ = 'aqeel'
import numpy as np
import os
from keras.models import load_model
from ALutils import RANK, Calculate_Score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
np.random.seed(1377)

Using TensorFlow backend.


In [2]:
FUll_Results = pd.read_csv('Results.csv')

### Prepare the data

In [2]:
train = pd.read_csv('../Data/train.csv')
ftest = pd.read_csv('../Data/test.csv')
#ftest[ftest.columns[[0]+[i for i in range(10,18)]]]
#train.head()
def GetData(ds):#, splitper=0.2): Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = 50 #int(math.floor(splitper * ds.shape[0] + 1))

    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    x_train = ds.iloc[splitper:][np.delete(ds.columns, ls)]
    y_train = ds.iloc[splitper:][ds.columns[10:18]]
    x_test = ds.iloc[:splitper][np.delete(ds.columns, ls)]
    y_test = ds.iloc[:splitper][ds.columns[10:18]]
    return (x_train.as_matrix(),y_train.as_matrix()),(x_test.as_matrix(),y_test.as_matrix()) 

def GetData2(ds):#, splitper=0.2): Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = 50 #int(math.floor(splitper * ds.shape[0] + 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    ds = scaler.fit_transform(ds)
    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    
    x_train = ds[splitper:, np.delete(range(ds.shape[1]), ls)]
    y_train = ds[splitper:, 10:18]
    x_test = ds[:splitper, np.delete(range(ds.shape[1]), ls)]
    y_test = ds[:splitper, 10:18]
    
    return (x_train,y_train),(x_test,y_test)


In [3]:
train.head()

,GeneID,"absolute expression, parental strain t=0 (arbitrary units)",wt t=0,wt t=10,wt t=20,wt t=30,wt t=45,wt t=60,wt t=90,wt t=120,...,tfB-del t=90,tfB-del t=120,tfC-del t=0,tfC-del t=10,tfC-del t=20,tfC-del t=30,tfC-del t=45,tfC-del t=60,tfC-del t=90,tfC-del t=120
0,45,0.194,0.0,0.080,-0.021,-0.208,0.071,-0.066,-0.064,-0.062,...,0.082,-0.080,-0.185,-0.205,-0.072,-0.061,0.000,0.042,-0.147,-0.246
1,7767,1.480,0.0,-0.467,-0.376,-0.619,-0.199,-0.107,-0.143,-0.210,...,0.302,-0.148,-0.149,-0.383,0.183,-0.007,0.020,-0.052,0.074,0.188
2,3443,13.960,0.0,-0.313,-0.147,-0.641,-0.098,-0.200,-0.305,-0.083,...,-0.478,-0.611,-0.385,-0.591,-0.320,-0.110,-0.469,-0.443,-0.285,-0.356
3,5965,132.600,0.0,-0.183,-0.106,-0.448,-0.119,-0.167,-0.245,-0.194,...,-0.302,-0.473,-0.285,-0.493,-0.358,-0.405,-0.500,-0.344,-0.263,-0.376
4,1341,0.148,0.0,0.165,0.183,0.186,0.103,0.130,0.108,0.186,...,0.112,0.349,0.037,0.013,-0.006,0.057,0.053,0.163,-0.024,-0.065


In [4]:
print('Loading data...')

#(X_train, y_train), (X_test, y_test) =GetData()
(x_train,y_train),(x_test,y_test) = GetData(train)
print('train:',x_train.shape,y_train.shape)
print('test: ',x_test.shape,y_test.shape)

Loading data...
train: (9235, 24) (9235, 8)
test:  (50, 24) (50, 8)


# Manual Testing
* Remember to select the data in the right shape for the model.

In [5]:
model = load_model('output_files/{}'.format(modelname))
print('Finished loading')
org = RANK(y_test)
pred = model.predict(x_test)
pred= RANK(pred)
predscore =Calculate_Score(pred,org)
orgscore = Calculate_Score(org,org)
print('C:{},P:{}'.format(predscore,orgscore))

NameError: name 'modelname' is not defined

# Test All Models Automatically on All the data
   * FUll_Results = pd.DataFrame(columns=['Model','Perfect Score','Predicted Score','Data Type'])

In [7]:
def Add_Info_DataSet(model,Predicted,Perfect,DataType):
    global FUll_Results
    FUll_Results = FUll_Results.append(pd.Series((model,Predicted,Perfect,DataType),index=['Model','Perfect Score','Predicted Score','Data Type']),ignore_index=True)

In [8]:
def StudyAModel(modelname):
    print('loading model')
    print(modelname)
    model = load_model('output_files/{}'.format(modelname))
    print('Finished loading')
    org = RANK(y_test)
    pred = model.predict(x_test)
    pred= RANK(pred)
    predscore =Calculate_Score(pred,org)
    orgscore = Calculate_Score(org,org)
    print('C:{},P:{}'.format(predscore,orgscore))
    Add_Info_DataSet(modelname,predscore,orgscore,'TEST')
    for i in range(0,184):
        x_tst = x_train[i*50:i*50+50]
        y_tst = y_train[i*50:i*50+50]
        org = RANK(y_tst)

        pred = model.predict(x_tst)
        pred = RANK(pred)
        predscore = Calculate_Score(pred,org)
        orgscore = Calculate_Score(org,org)
        if isinstance(predscore,str) or isinstance(orgscore,str):
            #problems.append('check{}, Score:{},{} '.format(i,str(predscore).strip(),str(orgscore).strip()))
            pass
        else:
            Add_Info_DataSet(modelname,predscore,orgscore,'TRN{}-{}'.format(i*50,i*50+50))
    print('Finished: {}'.format(modelname))

In [9]:
files=[]
for file in os.listdir("output_files/"):
    if file.endswith(".h5"):
        files.append(file)

In [10]:
#x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
#x_test = np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1], 1))
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1], 1))
for i in files:
    StudyAModel(i)

loading model
Model_LSTM9_00-0.09.h5
Finished loading
C:13.95448,P:33.16822
Finished: Model_LSTM9_00-0.09.h5
loading model
Model_LSTM9_05-0.07.h5
Finished loading
C:16.78712,P:33.16822
Finished: Model_LSTM9_05-0.07.h5
loading model
Model_LSTM9_02-0.07.h5
Finished loading
C:14.31508,P:33.16822
Finished: Model_LSTM9_02-0.07.h5
loading model
Model_LSTM9_24-0.06.h5
Finished loading
C:15.97445,P:33.16822
Finished: Model_LSTM9_24-0.06.h5
loading model
Model_LSTM9_17-0.06.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM9_17-0.06.h5
loading model
Model_LSTM9_04-0.07.h5
Finished loading
C:17.00778,P:33.16822
Finished: Model_LSTM9_04-0.07.h5
loading model
Model_LSTM9_13-0.06.h5
Finished loading
C:17.25942,P:33.16822
Finished: Model_LSTM9_13-0.06.h5
loading model
Model_LSTM9_10-0.06.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM9_10-0.06.h5
loading model
Model_LSTM9_03-0.07.h5
Finished loading
C:14.64455,P:33.16822
Finished: Model_LSTM9_03-0.07.h5


In [11]:
FUll_Results.to_csv('Results.csv',index=False)

In [4]:
FUll_Results[FUll_Results['Data Type']=='TEST'].head()
FUll_Results[FUll_Results['Data Type']=='TEST']['Perfect Score'].max()
FUll_Results[FUll_Results['Perfect Score']==18.96678]
#FUll_Results[FUll_Results['Model']=='Model_LSTM8_16-0.06.h5']

18.96678